# Get North American Mesoscale (NAM) temperature forecast

## Setup

In [1]:
import pandas as pd
import requests
import re
from datetime import datetime, timedelta
import time
import os
import pickle

from eccodes import *

import pymysql
# import boto3

## Hyperparameter

In [41]:
# denver international airport
LAT = 39.83
LON = -104.65

# nam download, now() is utc
# automation version
DATE = datetime.now().strftime('%Y%m%d')
# test version
# DATE = (datetime.now() - timedelta(days = 1)).strftime('%Y%m%d')
print(DATE)

PATH_NAM = 'https://nomads.ncep.noaa.gov/pub/data/nccf/com/nam/prod/'
PATH_DOWNLOAD = PATH_NAM + 'nam.' + DATE + '/'
print(PATH_DOWNLOAD)

PATTERN_FORECAST = 'nam\.t[0-9]{2}z\.awphys[0-9]{2}\.tm00\.grib2'
PATH_TMP = '/home/ubuntu/github/airflow-automation/01_notebooks/tmp'

MONITORING = 20
COLUMNS = ['name', 'latitude', 'longitude', 'level', 'typeOfLevel', 'dataDate', 'dataTime', 'validityDate', 'validityTime', 'value', 'units']

PATH_DF_01 = '/home/ubuntu/github/airflow-automation/01_notebooks/df_nam.pkl'

# mysql
HOST = 'ENDPOINT'
USER = 'USER'
PASSWORD = 'PASSWORD'
PORT = 3306
DB = 'DB'

# boto3

20200720
https://nomads.ncep.noaa.gov/pub/data/nccf/com/nam/prod/nam.20200720/


## Function

In [19]:
def nearest_four_to_one(nearest_four):
    dist_1 = nearest_four[0]['distance']
    dist_2 = nearest_four[1]['distance']
    dist_3 = nearest_four[2]['distance']
    dist_4 = nearest_four[3]['distance']
    val_1 = nearest_four[0]['value']
    val_2 = nearest_four[1]['value']
    val_3 = nearest_four[2]['value']
    val_4 = nearest_four[3]['value']
    one = (val_1 * 1 / dist_1 + val_2 * 1 / dist_2 + val_3 * 1 / dist_3 + val_4 * 1 / dist_4) / (1 / dist_1 + 1 / dist_2 + 1 / dist_3 + 1 / dist_4)
    return one

def get_temperature_from_grib(grib_file, lat, lon):
    lon_eccodes = lon + 360
    
    codes_grib_multi_support_on()
    
    f = open(grib_file, 'rb')
    
    while 1:
        gid = codes_grib_new_from_file(f)
        
        if gid is None:
            break
            
        name = codes_get(gid, 'name')
        level = codes_get(gid, 'level')
        dataDate = codes_get(gid, 'dataDate')
        dataTime = codes_get(gid, 'dataTime')
        validityDate = codes_get(gid, 'validityDate')
        validityTime = codes_get(gid, 'validityTime')
        typeOfLevel = codes_get(gid, 'typeOfLevel')
        units = codes_get(gid, 'units')
        
        if name == 'Temperature' and level == 0 and typeOfLevel == 'surface':
            nearest_four = codes_grib_find_nearest(gid, lat, lon_eccodes, is_lsm = False, npoints = 4)
            temp = nearest_four_to_one(nearest_four)
            
            return [name, lat, lon, level, typeOfLevel, dataDate, dataTime, validityDate, validityTime, temp, units]
        
        codes_release(gid)
        
    f.close()

## Extract data

In [20]:
start = time.time()
datas = []

# get all the grib file names of one day
r = requests.get(PATH_DOWNLOAD)
grib_file_names = re.findall(PATTERN_FORECAST, r.text)
grib_file_names = sorted(list(set(grib_file_names)))
print('Number of grib files', len(grib_file_names))
print(grib_file_names[0], grib_file_names[-1])

# from each grib file extract temperature data
for i, file in enumerate(grib_file_names):
    url = PATH_DOWNLOAD + file
    r = requests.get(url)
    f = open(PATH_TMP, 'wb').write(r.content)
    data = get_temperature_from_grib(PATH_TMP, LAT, LON)
    datas.append(data)
    
    # monitoring
    if i % MONITORING == 0:
        print('Stored {} results, {:.2f} minutes passed'.format(i, (time.time() - start) / 60))
        
print('Finished! Spent {:.2f} minutes'.format((time.time() - start) / 60))

Number of grib files 212
nam.t00z.awphys00.tm00.grib2 nam.t18z.awphys84.tm00.grib2
Stored 0 results, 0.31 minutes passed
Stored 20 results, 2.29 minutes passed
Stored 40 results, 3.81 minutes passed
Stored 60 results, 5.19 minutes passed
Stored 80 results, 7.05 minutes passed
Stored 100 results, 8.12 minutes passed
Stored 120 results, 9.12 minutes passed
Stored 140 results, 10.18 minutes passed
Stored 160 results, 11.31 minutes passed
Stored 180 results, 12.72 minutes passed
Stored 200 results, 13.74 minutes passed
Finished! Spent 15.76 minutes


In [33]:
df_nam = pd.DataFrame(datas, columns = COLUMNS)
df_nam['created_at'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

In [34]:
df_nam.shape

(212, 12)

In [35]:
df_nam.head()

,name,latitude,longitude,level,typeOfLevel,dataDate,dataTime,validityDate,validityTime,value,units,created_at
0,Temperature,39.83,-104.65,0,surface,20200718,0,20200718,0,310.358404,K,2020-07-20 00:24:17
1,Temperature,39.83,-104.65,0,surface,20200718,0,20200718,100,306.169533,K,2020-07-20 00:24:17
2,Temperature,39.83,-104.65,0,surface,20200718,0,20200718,200,302.576830,K,2020-07-20 00:24:17
3,Temperature,39.83,-104.65,0,surface,20200718,0,20200718,300,299.615338,K,2020-07-20 00:24:17
4,Temperature,39.83,-104.65,0,surface,20200718,0,20200718,400,296.861486,K,2020-07-20 00:24:17


In [36]:
df_nam.tail()

,name,latitude,longitude,level,typeOfLevel,dataDate,dataTime,validityDate,validityTime,value,units,created_at
207,Temperature,39.83,-104.65,0,surface,20200718,1800,20200721,1800,307.534087,K,2020-07-20 00:24:17
208,Temperature,39.83,-104.65,0,surface,20200718,1800,20200721,2100,306.153102,K,2020-07-20 00:24:17
209,Temperature,39.83,-104.65,0,surface,20200718,1800,20200722,0,297.471656,K,2020-07-20 00:24:17
210,Temperature,39.83,-104.65,0,surface,20200718,1800,20200722,300,290.793657,K,2020-07-20 00:24:17
211,Temperature,39.83,-104.65,0,surface,20200718,1800,20200722,600,288.961214,K,2020-07-20 00:24:17


## Save dataframe

In [37]:
pickle.dump(df_nam, open(PATH_DF_01, 'wb'))

## Upload RDS

In [38]:
df_nam.dtypes

name             object
latitude        float64
longitude       float64
level             int64
typeOfLevel      object
dataDate          int64
dataTime          int64
validityDate      int64
validityTime      int64
value           float64
units            object
created_at       object
dtype: object

In [39]:
val_to_insert = df_nam.values.tolist()
print('number of records inserted', len(val_to_insert))

# connection
connection = pymysql.connect(host = HOST,
                             user = USER,
                             password = PASSWORD,
                             port = PORT,
                             db = DB)

# query, adding for all the columns doesn't neet column names
query = """
INSERT INTO jupyterdb.nam_data
VALUES (%s, %s, %s, %s, %s, %s, 
        %s, %s, %s, %s, %s, %s)
"""

# execute
cursor = connection.cursor()
cursor.executemany(query, val_to_insert)
connection.commit()

number of records inserted 212


## Delete temporary file

In [40]:
if os.path.exists(PATH_TMP):
    print('The file exists')
    os.remove(PATH_TMP)
    
    if os.path.exists(PATH_TMP):
        print('Failed to delete')
    else:
        print('Success to delete')
else:
    print('The file does not exist')

The file exists
Success to delete


In [2]:
print(datetime.now())

2020-07-21 01:22:14.159325
